In [2]:
# Before running this program, download and setup ChromeDriver from https://googlechromelabs.github.io/chrome-for-testing/

!pip install selenium
from selenium import webdriver
import time
import re
import json

base_url = 'https://ubc.yul1.qualtrics.com'
lib_id = 'GR_cCIn1c2dzBm20Pc'
fdr_id = [
    {
        'FLDR_EYoJP6YyTXbey6g':35,
        'FLDR_G32abdVw1snafcR':35,
        'FLDR_Xk65II6Mlus1G1t':35,
        'FLDR_eSxJH8V3tmq2ser':35,
        'FLDR_a4wVLJRlemVuohU':37,
        'FLDR_iNZrmSue1uNJ6hT':37,
        'FLDR_TQJ9iY9tmeYhBRO':37,
        'FLDR_RlFgWJhXS64lRww':37,
        'FLDR_wGg4SMLfveevwCt':26,
        'FLDR_IVYwgYZah96s1Vx':26,
        'FLDR_2k0rcm2tPeSsTaB':26,
        'FLDR_03CbqzLmadteXmb':26,
        'FLDR_tLGwiIgD6YF4CcK':50,
        'FLDR_hKHGw3iBvZejZcl':50,
        'FLDR_cPiPFDxEgPZqeOq':50,
        'FLDR_tN0Leba92Kvmq8f':50,
        'FLDR_0Bdvn4ZGlhCZHnh':56,
        'FLDR_L0nYA5nGmvzzYe0':56,
        'FLDR_BU1ceU9kiMTlWOQ':56,
        'FLDR_Vn3cW8x5lejdqkV':56,
        'FLDR_oAGs0diDYV5Yqwt':55,
        'FLDR_eFOY7Uizj9MWKTS':55,
        'FLDR_DtaEVj9SSOPQz1v':55,
        'FLDR_rZcVSxVUZcFjarZ':55
    },
    {
        'FLDR_Xk65II6Mlus1G1t':35,
        'FLDR_eSxJH8V3tmq2ser':35,
        'FLDR_DeI1Ckc9ILqodWX':35,
        'FLDR_NoP0Jg5hEnVTCwK':35,
        'FLDR_TQJ9iY9tmeYhBRO':37,
        'FLDR_RlFgWJhXS64lRww':37,
        'FLDR_pFzsmDotlZJwxES':37,
        'FLDR_0muxunyS1BMF6QH':37,
        'FLDR_2k0rcm2tPeSsTaB':26,
        'FLDR_03CbqzLmadteXmb':26,
        'FLDR_du255eR83cO4xUS':26,
        'FLDR_jyk03NkhFitJpWd':26,
        'FLDR_cPiPFDxEgPZqeOq':50,
        'FLDR_tN0Leba92Kvmq8f':50,
        'FLDR_KczRShRgUoMZkNO':50,
        'FLDR_uRMZDX0nSXLPfoP':50,
        'FLDR_BU1ceU9kiMTlWOQ':56,
        'FLDR_Vn3cW8x5lejdqkV':56,
        'FLDR_zvciLcMHhW0NuRZ':56,
        'FLDR_GU3KSrsn7Sm0aQE':56,
        'FLDR_DtaEVj9SSOPQz1v':55,
        'FLDR_rZcVSxVUZcFjarZ':55,
        'FLDR_tybyvcBibRbKfu5':55,
        'FLDR_A57kRaIV8wR5pNJ':55
    }
]
login_sleep_time = 30  # Pause time for manual login process
pause_time = 5         # Temporary sleep time for every page processing
filename_pattern = r"<tr id=\"(.*?)\" aria-describedby=\"rbd-hidden-text-0-hidden-text-0\""
wav_pattern = r"<span class=\"elementNameLabel--_UZUPrXSkOhzyYK\">(\d+.*?)\.wav</span>"

stimuli = [{},{}]
dups = []
browser = webdriver.Chrome()
browser.get(base_url)
time.sleep(login_sleep_time)

for i in range(0,len(fdr_id)):
  for fdrid in fdr_id[i].keys():
    print(fdrid)
    for j in range(1, int((fdr_id[i][fdrid]+24)/25)+1):
      print(j)
      browser.get(base_url + '/app/library#libraryID=' + lib_id + '&folderID=' + fdrid + '&page=' + str(j))
      browser.refresh()
      time.sleep(pause_time)
      html_text = browser.page_source

      fn_matches = re.findall(filename_pattern, html_text)
      n_matches = re.findall(wav_pattern, html_text)
      for k in range(len(n_matches)):
        if n_matches[k] not in stimuli[i].keys():
          stimuli[i][n_matches[k]] = fn_matches[k]
        elif fn_matches[k] not in stimuli[i].values() and fn_matches[k] not in dups:
          dups.append(fn_matches[k])

browser.quit()

print("Found %d fast stimuli" % len(stimuli[0]))
print("Found %d slow stimuli" % len(stimuli[1]))
if dups:
    print("Wav file duplicates(can be deleted from library):")
    print(dups)
print(stimuli[0])
print(stimuli[1])


# HTML content of every question including two '%s' marks to be replaced of filename
js = "Listen to this rhythmic pattern, and rate the complexity of the rhythm.\n<div><br />\n<audio id=\"audio\" class=\"qmedia\" controls=\"true\" controlslist=\"noplaybackrate\"  height=\"40\" preload=\"auto\" width=\"320\"><source src=\"https://ubc.yul1.qualtrics.com/ControlPanel/File.php?F=%s\" type=\"audio/mp3\" /><embed class=\"qmedia\" flashconsts=\"file=https://ubc.yul1.qualtrics.com/ControlPanel/File.php?F=%s&amp;width=320&amp;height=20&amp;type=mp3&amp;autostart=false\" height=\"20\" pluginspage=\"http://adobe.com/flashplayer/\" src=\"/WRQualtricsShared/Flash/flvplayer.swf\" type=\"application/x-shockwave-flash\" width=\"320\" wmode=\"transparent\" /></audio>\n</div>"

# Key function for question sorting(first digit part of wav file)
def fn_digit(n):
    return int(re.findall(r'\d+', n)[0])

f = open('template.qsf')      # It's the simplified template file of 'Rhythmic_complexity_2024.qsf'
data = json.load(f)
f.close()

# SurveryElements[0]Payload[1:Fast, 2:Slow]

# i is the QuestionID number(QIDi start from 1 because QID0 already exists)
i = 1
for p in [1, 2]:
    ns = list(stimuli[p - 1].keys())
    ns.sort(key=fn_digit)

    for n in ns:
        # SurveyElements[9] is the QID0 question which is in Trash as the Question template
        zero = data['SurveyElements'][9].copy()
        # SurveryElements[0] is Survey Blocks, append questions to Payload[p]
        data['SurveyElements'][0]['Payload'][p]['BlockElements'].append({'Type': 'Question', 'QuestionID': 'QID'+str(i)})
        zero['PrimaryAttribute'] = 'QID' + str(i)
        zero['Payload'] = zero['Payload'].copy()
        zero['Payload']['QuestionText'] = js % (stimuli[p - 1][n], stimuli[p - 1][n])
        zero['Payload']['DataExportTag'] = n
        zero['Payload']['QuestionID'] = 'QID' + str(i)
        data['SurveyElements'].append(zero)
        i += 1

# Write the result to test2.qsf
f = open('Qualtrics Stimuli Questions.qsf', 'w')
json.dump(data, f, indent = 4)
f.close()

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/130.0.6723.58/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5b16ce49a0aa <unknown>
#1 0x5b16cdfb11a0 <unknown>
#2 0x5b16cdfe6faa <unknown>
#3 0x5b16cdfe2885 <unknown>
#4 0x5b16ce02dec6 <unknown>
#5 0x5b16ce02d516 <unknown>
#6 0x5b16ce021b23 <unknown>
#7 0x5b16cdff0990 <unknown>
#8 0x5b16cdff196e <unknown>
#9 0x5b16ce46616b <unknown>
#10 0x5b16ce469f68 <unknown>
#11 0x5b16ce45364c <unknown>
#12 0x5b16ce46aae7 <unknown>
#13 0x5b16ce4384af <unknown>
#14 0x5b16ce4884f8 <unknown>
#15 0x5b16ce4886c0 <unknown>
#16 0x5b16ce498f26 <unknown>
#17 0x7cee9c6d7ac3 <unknown>
